Faster lyrics extraction and emotion analysis

In [55]:
pip install openai-whisper ffmpeg pydub noisereduce numpy librosa

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [56]:
pip install tensorflow tf-keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [57]:
import os
import numpy as np
import librosa
import noisereduce as nr
import whisper
import soundfile as sf
from pydub import AudioSegment
from transformers import pipeline
import textwrap
import torch
import threading

In [58]:
#Faster
def convert_mp3_to_wav(mp3_path, wav_path):
    """Convert MP3 to WAV format."""
    audio = AudioSegment.from_mp3(mp3_path)
    audio = audio.set_channels(1).set_frame_rate(16000)  # Convert to mono & 16kHz for better accuracy
    audio.export(wav_path, format="wav")

def remove_background_noise(wav_path, output_path, fast=True):
    """Load WAV file, reduce noise, and save cleaned audio."""
    y, sr = librosa.load(wav_path, sr=16000)

    if fast:
        # Noise reduction skipped
        sf.write(output_path, y, sr)
    else:
        # Noise reduction performed
        reduced_noise = nr.reduce_noise(y=y, sr=sr, prop_decrease=0.5)
        sf.write(output_path, reduced_noise, sr)

def transcribe_with_whisper(wav_path):
    """Transcribe lyrics using OpenAI's Whisper model with GPU (if available)."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = whisper.load_model("base").to(device)
    result = model.transcribe(wav_path)
    return result["text"]

def extract_lyrics_from_mp3(mp3_path, fast_mode=True):
    """Extract lyrics from an MP3 song file quickly."""
    wav_path = "temp.wav"
    cleaned_wav_path = "cleaned_temp.wav"

    # MP3 to wav
    convert_thread = threading.Thread(target=convert_mp3_to_wav, args=(mp3_path, wav_path))
    convert_thread.start()
    convert_thread.join()

    # Background noise remove
    remove_background_noise(wav_path, cleaned_wav_path, fast=fast_mode)

    lyrics = transcribe_with_whisper(cleaned_wav_path)

    # Temporary file removal
    os.remove(wav_path)
    os.remove(cleaned_wav_path)

    return lyrics

mp3_file = "Mockingbird.mp3"  #Song file name
lyrics = extract_lyrics_from_mp3(mp3_file, fast_mode=True)
print("Extracted Lyrics:\n", lyrics)

Extracted Lyrics:
  I know sometimes things may not always make sense to you right now, but hey, I know you miss your mom and I know you miss your dad when I'm gone, but I'm trying to give you the life that I never had. I can see you sad even when you smile, even when you laugh, I can see it in your eyes deep inside you want to cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't cry, I can't Daddy, daddy had to see Daddy don't want you to see But you see just as much as he di

In [59]:
#1 (POSITIVE/NEGATIVE)

sentiment_pipeline = pipeline("sentiment-analysis")

# Split into 500-character chunks
chunks = textwrap.wrap(lyrics, width=500)

results = [sentiment_pipeline(chunk, truncation=True)[0] for chunk in chunks]
print(results)

# Aggregate results (count positive/negative instances)
positive = sum(1 for r in results if r["label"] == "POSITIVE")
negative = sum(1 for r in results if r["label"] == "NEGATIVE")

overall_sentiment = "POSITIVE" if positive > negative else "NEGATIVE"
print("Overall Sentiment:", overall_sentiment)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9980165958404541}, {'label': 'POSITIVE', 'score': 0.9603163003921509}, {'label': 'NEGATIVE', 'score': 0.9960816502571106}, {'label': 'NEGATIVE', 'score': 0.7300128936767578}, {'label': 'NEGATIVE', 'score': 0.9992552399635315}, {'label': 'NEGATIVE', 'score': 0.9991182684898376}, {'label': 'POSITIVE', 'score': 0.9909221529960632}, {'label': 'POSITIVE', 'score': 0.8762101531028748}, {'label': 'POSITIVE', 'score': 0.9996156692504883}]
Overall Sentiment: POSITIVE


In [60]:
#2 (POSITIVE/NEGATIVE)

sentiment_pipeline = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

# Split into 500-character chunks
chunks = textwrap.wrap(lyrics, width=500)

# Analysis of each chunk
results = [sentiment_pipeline(chunk)[0] for chunk in chunks]

print(results)

# Aggregate sentiment scores
positive = sum(1 for r in results if r["label"] == "positive")
negative = sum(1 for r in results if r["label"] == "negative")
neutral = sum(1 for r in results if r["label"] == "neutral")

# Overall sentiment
if positive > negative:
    overall_sentiment = "POSITIVE"
elif negative > positive:
    overall_sentiment = "NEGATIVE"
else:
    overall_sentiment = "NEUTRAL"

print("Overall Sentiment:", overall_sentiment)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'label': 'negative', 'score': 0.5577308535575867}, {'label': 'negative', 'score': 0.7909740209579468}, {'label': 'negative', 'score': 0.8046436309814453}, {'label': 'positive', 'score': 0.5788432359695435}, {'label': 'negative', 'score': 0.7715982794761658}, {'label': 'negative', 'score': 0.5956785082817078}, {'label': 'negative', 'score': 0.44218334555625916}, {'label': 'positive', 'score': 0.759391188621521}, {'label': 'positive', 'score': 0.8953444957733154}]
Overall Sentiment: NEGATIVE


In [61]:
#3 (Emotions)

sentiment_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

# Split lyrics into smaller parts of 500 characters
chunks = textwrap.wrap(lyrics, width=500)

# Analysis of each chunk
results = [sentiment_pipeline(chunk)[0] for chunk in chunks]

print(results)

Device set to use cpu


[{'label': 'sadness', 'score': 0.9617969989776611}, {'label': 'sadness', 'score': 0.6956216096878052}, {'label': 'neutral', 'score': 0.36514657735824585}, {'label': 'surprise', 'score': 0.7306612730026245}, {'label': 'sadness', 'score': 0.9416980147361755}, {'label': 'sadness', 'score': 0.2939087152481079}, {'label': 'surprise', 'score': 0.9262656569480896}, {'label': 'sadness', 'score': 0.3168991506099701}, {'label': 'joy', 'score': 0.5044536590576172}]
